In [2]:
#数値計算ライブラリを追加
using LinearAlgebra

In [3]:
#各変数を宣言
hc_mevfm = 197.327
mαc2_mev = 3727.379
e2       = 1.44
Z        = 84
Z_prime  = 82
Zα       = 2
V0_mev   = 50
R_fm     = 8.0
Δr_fm    = 0.5
N        = 400
cΔt_fm   = 0.2
Nexp     = 4
VB_mev       = mαc2_mev * Zα * Z / R_fm

78274.959

In [4]:
#Hswの作成
Hsw = zeros(N,N)
mom_coeff = - hc_mevfm^2 /(2 * mαc2_mev * Δr_fm^2)
#対角成分の計算
for i in 1:N
    potential_term = (i * Δr_fm < R_fm) ? -V0_mev : VB_mev
    Hsw[i,i] = -2 * mom_coeff + potential_term
end

#非対角成分の計算
for i in 1:N-1
    Hsw[i,i+1] = mom_coeff
    Hsw[i+1,i] = mom_coeff
end


In [5]:
#Hswの対角化
ϵ,ψ = eigen(Symmetric(Hsw))


Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
400-element Vector{Float64}:
   -49.197122703787045
   -46.81934192440769
   -42.95802585700406
   -37.761549860348254
   -31.42959642984127
   -24.205483818280833
   -16.366817760103906
    -8.214825196897728
    -0.062779605031551
     7.776037465834868
     ⋮
 78358.41869212108
 78358.44220063945
 78358.46294711517
 78358.480930174
 78358.49614862475
 78358.50860145933
 78358.51828785286
 78358.52520716372
 78358.52935893359
vectors:
400×400 Matrix{Float64}:
 0.0689732   0.135296    -0.19642    …   0.0          1.43189e-57
 0.135296    0.249995    -0.326636       0.0         -5.37084e-54
 0.196419    0.326636    -0.346759       0.0          2.01453e-50
 0.249995    0.353551    -0.250008       0.0         -7.55626e-47
 0.293963    0.326643    -0.0689904      0.0          2.83426e-43
 0.326635    0.250008     0.13528    …   0.0         -1.06309e-39
 0.346756    0.135313     0.293954       0.0          3.98753e-36
 0.353

In [6]:
#正のエネルギー固有値を持つ束縛状態の数をカウント
count = 0
energy_eigenvalues = []
for i in 1:N
    if VB_mev > ϵ[i] > 0
        count += 1
        push!(energy_eigenvalues,ϵ[i])
    end
end
println("Number of bound states: ", count)
println("Energy eigenvalues: ", energy_eigenvalues)

Number of bound states: 6
Energy eigenvalues: Any[7.776037465834868, 15.000376083851009, 21.33259610721292, 26.529338696730818, 30.39088077058456, 32.768812562671954]


In [7]:
#Hsw+cの作成
Hsw_c = zeros(N,N)

#Hswの作成
Hsw = zeros(N,N)
mom_coeff = - hc_mevfm^2 /(2 * mαc2_mev * Δr_fm^2)
#対角成分の計算
for i in 1:N
    if i * Δr_fm < R_fm
        potential_term = -V0_mev
    else
        potential_term = e2 * Z_prime * Z / (i * Δr_fm)
        
    end

    Hsw[i,i] = -2 * mom_coeff + potential_term
end

#非対角成分の計算
for i in 1:N-1
    Hsw[i,i+1] = mom_coeff
    Hsw[i+1,i] = mom_coeff
end

In [ ]:
function calc_next_evolution(ψ, H, N, cΔt_fm, Nexp)
    ψ_next = zeros(N)
    for k in 0:Nexp
        ψ_next += 1/factorial(k) * (cΔt_fm / im / hc_mevfm)^k * (H^k * ψ)
    end
    return ψ_next
end

calc_next_evolution (generic function with 1 method)

In [ ]:
function calc_time_evolution(ψ, H)

i: 0
i: 1
i: 2
i: 3
i: 4
